In [2]:
import cv2
import os
import numpy as np
from random import shuffle
import imutils #need to install this with pip
import matplotlib.pyplot as plt
from skimage import data, color, exposure
from sklearn import svm
import data_generator as dg
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline
import glob

ModuleNotFoundError: No module named 'imutils'